Okay! Let's try the gym business. Cliff walking environment is perfect.

In [1]:
import gym
import numpy as np
import ToyEnv
import psrl
import PosteriorSamplerGMM

In [2]:
env = gym.make('CliffWalking-v0')

In [3]:
observation, info = env.reset()
print(observation)
print(info)
np.unravel_index(observation, env.shape)

36
{'prob': 1}


(3, 0)

In [4]:
action = env.action_space.sample()
print(action)
observation, reward, terminated, truncated, info = env.step(3)
print(observation)
print(env.action_space.n)
print(reward)
print(env.P[observation][2])

2
36
4
-1
[(1.0, 36, -1, False)]


/home/srayan/miniconda3/envs/tsmdp/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Testing ToyEnv

In [15]:
new_env = ToyEnv.ToyEnv(3, 3, 2)
init_state = new_env.reset()
print(init_state)

0


In [5]:
next_state, reward = new_env.step(1)
print(reward)
print(next_state)

3.937980259205279
0


# Testing Gibbs sampler

In [4]:
# Generate some data for state-action pair (0, 2)
means = new_env._compute_means(0, 2)
sigma = new_env.true_Sigma
pis = new_env.true_pi
data = np.zeros((100,))
for i in range(100):
    k = np.random.choice(len(pis), p=pis)
    data[i] = np.random.normal(loc=means[k], scale=sigma[k])

In [17]:
print(means)
print(sigma)
print(pis)

[2 5]
[0.1 0.1]
[0.2 0.8]


In [22]:
gibbs = PosteriorSamplerGMM.GibbsSamplerGMM(1000, 2)

In [23]:
samples = gibbs.fit(data)

In [25]:
print(samples[-500])

(array([5.00015311, 1.9834373 ]), array([0.06634279, 0.17205711]), array([0.82502349, 0.17497651]))


# Testing PSRL

In [1]:
import VariationalInference

In [16]:
alg = psrl.PSRL(env=new_env, T = 1)


In [17]:
_, _, policy = alg.run()

Iteration: 0
Episode 0 Regret: 3.390119297318861


In [18]:
d = alg.to_dict()

In [19]:
import pickle

In [23]:
f = open(b"test_pickle.obj", "wb")
pickle.dump(d, f)

In [24]:
l = pickle.load(open(b"test_pickle.obj", "rb"))

In [25]:
print(l)

{'H': 100, 'T': 1, 'policy': array([[2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
        2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,

In [16]:
import dill

ModuleNotFoundError: No module named 'dill'

In [4]:
import json
sample_dict = {"Gibbs": {"state": {2: 4.5, 3: 7}}, "HMC": {"state": {2: 4.5, 3: 7}}}
with open("time.json", "w") as f:
    json.dump(sample_dict, f)

In [6]:
sample_dict["Gibbs"]["state"][2] = 5

In [7]:
print(sample_dict)

{'Gibbs': {'state': {2: 5, 3: 7}}, 'HMC': {'state': {2: 4.5, 3: 7}}}


In [2]:
import VariationalInference

In [7]:
new_env = ToyEnv.ToyEnv(3, 3, 2)
init_state = new_env.reset()
print(init_state)

1


In [8]:
alg = psrl.PSRL(env=new_env,sampler=VariationalInference.VIpymcGMM)
_, _, policy = alg.run()

Iteration: 0


AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
import pymc as pm
n_components = 3

In [10]:
x_1 = np.random.normal(loc=0, scale=1, size=100)
z = np.random.choice(n_components, size=100)
x = np.zeros(100)
x[z == 0] = np.random.normal(loc=0, scale=1, size=(z == 0).sum())
x[z == 1] = np.random.normal(loc=5, scale=1, size=(z == 1).sum())
x[z == 2] = np.random.normal(loc=10, scale=1, size=(z == 2).sum())

In [12]:
mu0 = 0
sigma0 = 1
alphaG = 1
betaG = 0.5

In [14]:
with pm.Model(coords={"cluster" : range(n_components)}) as some_model:
    mu = pm.Normal("mu",
                            mu = mu0, 
                            sigma = sigma0,
                            dims="cluster"
                            )
    sigma = pm.InverseGamma("sigma",
                                 beta = betaG,
                                 alpha = alphaG,
                                 dims = "cluster")
    weights = pm.Dirichlet("w", np.ones(n_components), dims="cluster")
    obs = pm.NormalMixture("obs", w=weights, mu=mu, sigma=sigma, observed=x)

In [19]:
with some_model:
    mean_field = pm.fit(obj_optimizer=pm.adagrad_window(learning_rate=0.01), progressbar=False)

/home/srayan/miniconda3/envs/608proj/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Finished [100%]: Average Loss = 291.95


In [16]:
sanokes = mean_field.sample(1000)

In [17]:
sanokes.posterior["mu"]

<xarray.DataArray 'mu' (chain: 1, draw: 1000, cluster: 3)> Size: 24kB
array([[[ 5.25501492, -0.30676995,  0.29734657],
        [ 4.60955132, -0.66414261,  0.16030605],
        [ 5.46152895,  0.02220216, -0.08072968],
        ...,
        [ 4.21003559, -0.16430032,  0.3279469 ],
        [ 5.45115943, -0.46497134,  0.37075777],
        [ 4.43654111, -0.32639393, -0.26803871]]])
Coordinates:
  * chain    (chain) int64 8B 0
  * draw     (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * cluster  (cluster) int64 24B 0 1 2

In [2]:
import numpy as np

In [8]:
env = ToyEnv.ToyEnv(3, 3, 2, seed = 3)

In [4]:
print(env.true_pi)

[0.2 0.8]


In [9]:
env.reset()

0

In [10]:
stuff = env.to_dict()

In [26]:
f = open(b"test_pickle.obj", "wb")
pickle.dump(stuff, f)

In [27]:
l = pickle.load(open(b"test_pickle.obj", "rb"))